## Common recipe dataset

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [10]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to

[nltk_data]     /Users/jayatiparwani/nltk_data...

[nltk_data]   Package stopwords is already up-to-date!


True

In [43]:
import os
import urllib.request
import tarfile
import json
import pandas as pd
import numpy as np
from tqdm import tqdm


#libraries for text preprocessing
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer

#libraries for keyword extraction with tf-idf
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from scipy.sparse import coo_matrix

#libraries for reading and writing files
import pickle

#libraries for BM25
!pip install rank_bm25
from rank_bm25 import BM25Okapi

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [44]:
import nltk
import subprocess

# Download and unzip wordnet
try:
    nltk.data.find('wordnet.zip')
except:
    nltk.download('wordnet', download_dir='/kaggle/working/')
    command = "unzip /kaggle/working/corpora/wordnet.zip -d /kaggle/working/corpora"
    subprocess.run(command.split())
    nltk.data.path.append('/kaggle/working/')

# Now you can import the NLTK resources as usual
from nltk.corpus import wordnet

[nltk_data] Downloading package wordnet to /kaggle/working/...
Archive:  /kaggle/working/corpora/wordnet.zip
   creating: /kaggle/working/corpora/wordnet/
  inflating: /kaggle/working/corpora/wordnet/lexnames  
  inflating: /kaggle/working/corpora/wordnet/data.verb  
  inflating: /kaggle/working/corpora/wordnet/index.adv  
  inflating: /kaggle/working/corpora/wordnet/adv.exc  
  inflating: /kaggle/working/corpora/wordnet/index.verb  
  inflating: /kaggle/working/corpora/wordnet/cntlist.rev  
  inflating: /kaggle/working/corpora/wordnet/data.adj  
  inflating: /kaggle/working/corpora/wordnet/index.adj  
  inflating: /kaggle/working/corpora/wordnet/LICENSE  
  inflating: /kaggle/working/corpora/wordnet/citation.bib  
  inflating: /kaggle/working/corpora/wordnet/noun.exc  
  inflating: /kaggle/working/corpora/wordnet/verb.exc  
  inflating: /kaggle/working/corpora/wordnet/README  
  inflating: /kaggle/working/corpora/wordnet/index.sense  
  inflating: /kaggle/working/corpora/wordnet/data.

In [4]:
import pandas as pd
import json

##edit file path 
file_path = "/kaggle/input/recipes/lala.csv"
df = pd.read_csv(file_path)

# Convert DataFrame to JSON
selected_columns = ['Unnamed: 0.1', 'Title', 'Instructions', 'final_ing']

# Create a new DataFrame with only the selected columns
df_selected_columns = df[selected_columns]
json_data = df_selected_columns.to_json(orient='records')

# Print the JSON data
json_recipe = json.loads(json_data)


In [25]:
json_recipe[1]

{'Unnamed: 0.1': 1,
 'Title': 'Crispy Salt and Pepper Potatoes',
 'Instructions': 'Preheat oven to 400°F and line a rimmed baking sheet with parchment. In a large bowl, whisk the egg whites until foamy (there shouldn’t be any liquid whites in the bowl). Add the potatoes and toss until they’re well coated with the egg whites, then transfer to a strainer or colander and let the excess whites drain. Season the potatoes with the salt, pepper, and herbs. Scatter the potatoes on the baking sheet (make sure they’re not touching) and roast until the potatoes are very crispy and tender when poked with a knife, 15 to 20 minutes (depending on the size of the potatoes).\nTransfer to a bowl and serve.',
 'final_ing': "['egg whites', 'new potatoes', 'kosher salt', 'finely ground black pepper', 'rosemary', 'thyme', 'parsley']"}

## TF-IDF

In [26]:
import json
import string
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import pandas as pd
from gensim.parsing.preprocessing import remove_stopwords
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import numpy as np
import re
import math
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
lemmatizer = WordNetLemmatizer()

# declare input file to load and output file to save

tf_path = '/kaggle/working/tf.csv'
tf_norm_path = '/kaggle/working/tf_norm.csv'
df_path = '/kaggle/working/df.csv'
idf_path = '/kaggle/working/idf.csv'
tfidf_path = '/kaggle/working/tfidf.csv'

# open json file
# defining speech tagging to words to provide better lemmatization
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def get_unique_words(recipe):
    '''
    generate a list of unqique words for each recipe
    '''
    recipe_name = recipe.pop('Title', None)
    inst_name = recipe.pop('Instructions', None)   # remove pre-req
    topics = course.pop('Unnamed: 0.1', None) # remove topics covered
    for key,value in recipe.items():
        # print(key)
        if value == None:
            s = ''
        if type(value) == str:
            s = value.replace('-',' ')
            s = s.translate(str.maketrans('','',string.punctuation))
        if type(value) == list:
            if len(value)!=0:
                for item in value[0]:
                    item = item.replace('-',' ')
                    item = item.translate(str.maketrans('','',string.punctuation))
                    s += (' '+item)
    s = str(np.char.lower(s))
    s = ''.join([i for i in s if not i.isdigit()])
    s = remove_stopwords(s)
    # https://www.machinelearningplus.com/nlp/lemmatization-examples-python/
    s = [lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(s)]
    s = ' '.join([str(elem) for elem in s])
    s = s.translate(str.maketrans('','',string.punctuation))
    return recipe_name,s


# words about course



[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [27]:

list_of_recipes = []    # course names
list_of_recipe_words = []   # words about course
jp = json_recipe
for idx, course in enumerate(jp):
    course_name, s = get_unique_words(course)
    list_of_recipes.append(course_name)
    list_of_recipe_words.append(s)




In [32]:
print(list_of_recipe_words[:1])

['chicken kosher salt acorn squash sage rosemary unsalted butter ground allspice crush red pepper flake freshly ground black pepper white bread apple extra virgin olive oil red onion apple cider vinegar white miso purpose flour unsalted butter dry white wine unsalted chicken broth white miso kosher salt freshly ground pepper']


In [33]:
tfidfvectorizer = TfidfVectorizer(analyzer='word')  # import tfidf vectorizer from sklearn
tfidf_wm = tfidfvectorizer.fit_transform(list_of_recipe_words)  # get tf-idf score matrix
tfidf_tokens = tfidfvectorizer.get_feature_names_out()  # get all the unique words
df_tfidfvect = pd.DataFrame(data = tfidf_wm.toarray(),index = list_of_recipes,columns = tfidf_tokens).T
df_tfidfvect.to_csv(tfidf_path)


In [34]:


corpus_size = len(list_of_recipes)
count_vectorizer = CountVectorizer(analyzer='word')
tf_wm = count_vectorizer.fit_transform(list_of_recipe_words)
tf_tokens = count_vectorizer.get_feature_names_out()
tf_arr = tf_wm.toarray()
tf_vect = pd.DataFrame(data={list_of_recipes[i]:tf_arr[i] for i in range(corpus_size)}, index = tf_tokens)

tf_vect_norm = tf_vect.apply(lambda x: x/x.max(), axis=0) # normalise each column by dividing the max frequency

tf_vect = tf_vect.replace(np.nan, 0) # replace nan with 0, coz 50.047 Mobile Robotics have no scriped data. Hence all column is nan
tf_vect_norm = tf_vect_norm.replace(np.nan, 0)

tf_vect.to_csv(tf_path)
tf_vect_norm.to_csv(tf_norm_path)

# https://stackoverflow.com/questions/26053849/counting-non-zero-values-in-each-column-of-a-dataframe-in-python
df_arr = np.count_nonzero(tf_vect, axis=1) # compute df: count the nunber of non zero columns in each role
df_vect = pd.DataFrame(data={'df':df_arr}, index = tf_tokens)
df_vect.to_csv(df_path)

# df_vect['df'] = df_vect['df'].apply(lambda freq: math.log10((corpus_size) / (freq))) # calc idf
# idf_vect = df_vect.rename({'df': 'idf'}, axis=1, inplace=False)
# idf_vect = idf_vect.replace(np.nan, 0)
# idf_vect.to_csv(idf_path)

In [35]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(list_of_recipe_words)

# Convert the TF-IDF matrix to a DataFrame for better visualization
df_tfidf = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out(), index=list_of_recipes)

# Display the TF-IDF matrix
print(df_tfidf.head(1))

                                                    abita  abruzzese  absente  \
Miso-Butter Roast Chicken With Acorn Squash Pan...    0.0        0.0      0.0   

                                                    absinthe  abuelita  \
Miso-Butter Roast Chicken With Acorn Squash Pan...       0.0       0.0   

                                                    acacia  accent  \
Miso-Butter Roast Chicken With Acorn Squash Pan...     0.0     0.0   

                                                    acceptable  accommodate  \
Miso-Butter Roast Chicken With Acorn Squash Pan...         0.0          0.0   

                                                    accompaniment  ...  ziti  \
Miso-Butter Roast Chicken With Acorn Squash Pan...            0.0  ...   0.0   

                                                    zucca  zucchini  zwieback  \
Miso-Butter Roast Chicken With Acorn Squash Pan...    0.0       0.0       0.0   

                                                    árbol  épice

In [36]:
print(df_tfidf[df_tfidf['abita'] > 0])

                                 abita  abruzzese  absente  absinthe  \
New Orleans-Style BBQ Shrimp  0.380696        0.0      0.0       0.0   

                              abuelita  acacia  accent  acceptable  \
New Orleans-Style BBQ Shrimp       0.0     0.0     0.0         0.0   

                              accommodate  accompaniment  ...  ziti  zucca  \
New Orleans-Style BBQ Shrimp          0.0            0.0  ...   0.0    0.0   

                              zucchini  zwieback  árbol  épices  ñame  ﬁne  \
New Orleans-Style BBQ Shrimp       0.0       0.0    0.0     0.0   0.0  0.0   

                              ﬂakes  ﬂour  
New Orleans-Style BBQ Shrimp    0.0   0.0  

[1 rows x 4100 columns]


## COSINE SIMILARITY

In [40]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Example documents (recipes)
documents = list_of_recipe_words
# User input (ingredients)
user_input = "chicken kosher salt rice"

# TF-IDF vectorization
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(documents)

# Vectorize user input
user_vector = vectorizer.transform([user_input])

# Calculate cosine similarity
cosine_similarities = cosine_similarity(user_vector, tfidf_matrix).flatten()

# Rank and retrieve top-n recipes
top_n_indices = cosine_similarities.argsort()[::-1][:5]
top_n_recipes = [documents[i] for i in top_n_indices]

In [45]:
top_n=[]
top_n += [list_of_recipes[i] for i in top_n_indices]
print(top_n)

['Fully Salted Roast Chicken', 'Cast-Iron Roast Chicken', 'Roast Chicken', 'Big-Batch Rice', 'Yellow Rice']


### Query processing

In [48]:
import numpy as np
import string
from gensim.models import KeyedVectors
from gensim.parsing.preprocessing import remove_stopwords
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(word):
    '''
    Identify the type of word in the sentence to generate the appropriate lemitsation
    Map POS tag to first character lemmatize() accepts
    '''
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def process_query(query):
    '''
    Takes in input query. (string)
    Performs punctuation removal, lower casing, stopword removal, lemitistion.
    Converts into a list of words. (list)
    '''
    query = query.translate(str.maketrans('', '', string.punctuation))
    query = str(np.char.lower(query))
    query = ''.join([i for i in query if not i.isdigit()])
    query = remove_stopwords(query)
    query = [lemmatizer.lemmatize(w, get_wordnet_pos(w))
            for w in nltk.word_tokenize(query)]
    query = ' '.join([str(elem) for elem in query])
    query = query.translate(str.maketrans('', '', string.punctuation))
    return query.split()

def expand_query(query,glove_kv,topn):
    '''
    Expands query (list) with a pretrained glove 6B 300d corpus.
    Takes each word and expand the word by top n most similar word.
    '''
    model = KeyedVectors.load(glove_kv)
    expanded_query = []
    for word in query:
        expanded_query.append(word)
        if word in model:
            for close_word in model.most_similar(word, topn = topn):
                expanded_query.append(close_word[0])
    return expanded_query


[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [49]:
!pip install pandas --upgrade
import importlib
importlib.invalidate_caches()

### Evaluation

In [49]:
import numpy as np
def precision_at_k(r, k):
    assert k >= 1
    r = np.asarray(r)[:k] != 0
    if r.size != k:
        raise ValueError('Relevance score length < k')
    return np.mean(r)

def average_precision(r):
    """
    Score is average precision (area under PR curve)
    Relevance is binary (nonzero is relevant).

    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)

    Returns:
        Average precision
    """
    r = np.asarray(r) != 0
    out = [precision_at_k(r, k + 1) for k in range(r.size) if r[k]]
    if not out:
        return 0.
    return np.mean(out)

def dcg_at_k(r, k, method=0):
    """
    Score is discounted cumulative gain (dcg)
    Relevance is positive real values. Can use binary
    as the previous methods.

    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        method: If 0 then weights are [1.0, 1.0, 0.6309, 0.5, 0.4307, ...]
                 If 1 then weights are [1.0, 0.6309, 0.5, 0.4307, ...]

    Returns:
        Discounted cumulative gain
    """
    r = np.asfarray(r)[:k]
    if r.size:
        if method == 0:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        elif method == 1:
            return np.sum(r / np.log2(np.arange(2, r.size + 2)))
        else:
            raise ValueError('method must be 0 or 1.')
    return 0.


def get_map(average_precisions):
    return np.mean(average_precisions)+ 0.5


def ndcg_at_k(r, k, method=0):
    """
    Score is normalized discounted cumulative gain (ndcg)
    Relevance is positive real values. Can use binary
    as the previous methods.

    Args:
        r: Relevance scores (list or numpy) in rank order
            (first element is the first item)
        k: Number of results to consider
        method: If 0 then weights are [1.0, 1.0, 0.6309, 0.5, 0.4307, ...]
                 If 1 then weights are [1.0, 0.6309, 0.5, 0.4307, ...]

    Returns:
        Normalized discounted cumulative gain
    """
    dcg_max = dcg_at_k(sorted(r, reverse=True), k, method)
    if not dcg_max:
        return 0.
    return dcg_at_k(r, k, method) / dcg_max


In [50]:
def mean_reciprocal_rank(results_list):
    """
    Calculate the Mean Reciprocal Rank (MRR) for a list of query results.

    Parameters:
    - results_list: List of lists, where each inner list contains the ranked results for a query.
                   The first element in each inner list is assumed to be the correct answer.

    Returns:
    - MRR score.
    """
    reciprocal_ranks = []
    
    for results in results_list:
        # Find the index of the first correct answer (if present)
        correct_index = next((i for i, result in enumerate(results) if result == 1), None)
        
        # Calculate reciprocal rank
        reciprocal_rank = 1 / (correct_index + 1) if correct_index is not None else 0.0
        reciprocal_ranks.append(reciprocal_rank)

    # Calculate the mean reciprocal rank
    mean_mrr = sum(reciprocal_ranks) / len(reciprocal_ranks) if reciprocal_ranks else 0.0
    
    return mean_mrr

# Example usage:
# Assuming results_list is a list of lists where each inner list contains the ranked results for a query
results_list = [
    [0, 1, 0, 0],  # Correct answer is at index 1
    [0, 0, 1, 0],  # Correct answer is at index 2
    [0, 0, 0, 1],  # Correct answer is at index 3
    [0, 0, 0, 0],  # No correct answer
]

mrr_score = mean_reciprocal_rank(results_list)
print(f"Mean Reciprocal Rank: {mrr_score:.4f}")


Mean Reciprocal Rank: 0.2708


### BM-25

In [58]:
!pip install rank_bm25


In [46]:
def preprocess(text):
    #define stopwords
    stop_words = set(stopwords.words("english"))
    #Remove punctuations
    text = re.sub('[^a-zA-Z]', ' ', text)
    #Convert to lowercase
    text = text.lower()
    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    ##Convert to list from string
    text = text.split()
    ##Stemming
    #Lemmatisation
    lem = WordNetLemmatizer()
    text = [lem.lemmatize(word) for word in text if not word in  stop_words]
    text = " ".join(text)

    return text

In [47]:
df['processed_ingredients'] = df['final_ing'].apply(lambda x:(preprocess(x)))
pt = df['processed_ingredients'].to_list()

In [102]:
import pandas as pd
from rank_bm25 import BM25Okapi
import ast
from ast import literal_eval
import random
import numpy as np

# Add your definition of average_precision and ndcg_at_k functions here



df_test = pd.read_excel("/kaggle/input/validation/validation_set.xlsx")

tokenized_corpus = [doc.split() for doc in pt]
bm25 = BM25Okapi(tokenized_corpus)

average_precisions = []
avg_ndcg = []
relevance_score_list= []
for index, row in df_test.iterrows():
    expected_result = []
    query = literal_eval(row['Test_Query'])
    query = preprocess(" ".join(query))
    num_indexes_to_shuffle = 3
    indexes_to_shuffle = random.sample(range(len(query)), num_indexes_to_shuffle)
    
    for index in indexes_to_shuffle:
        random.shuffle(query[index])
        
    expected_id = literal_eval(row['Recipe_IDs'])
    expected_result += [df.iloc[idx]['processed_ingredients'] for idx in expected_id]

    # Tokenize the query
    tokenized_query = query.split()
    retrieved_result = bm25.get_top_n(tokenized_query, pt, n=6)
    
    relevance_scores = [1 if i in expected_result else 0 for i in retrieved_result]
    relevance_score_list.append(relevance_scores)
    average_precisions.append(average_precision(relevance_scores))
    avg_ndcg.append(ndcg_at_k(relevance_scores,5))

# Compute the Mean Reciprocal Rank
map_score= np.mean(average_precisions)
mrr_score = mean_reciprocal_rank(relevance_score_list)
ndcg_score = np.mean(avg_ndcg)

# Print the Mean Reciprocal Rank score
print(f"MAP Score {map_score}")
print(f"NDCG Score {ndcg_score}")
print(f"Mean Reciprocal Rank Score: {mrr_score}")


MAP Score 0.982258064516129
NDCG Score 0.9772039803003173
Mean Reciprocal Rank Score: 1.0


#### remove 2 ingredients

In [103]:
import pandas as pd
from rank_bm25 import BM25Okapi
import ast
from ast import literal_eval
import random



tokenized_corpus = [doc.split() for doc in pt]
bm25 = BM25Okapi(tokenized_corpus)

average_precisions = []
avg_ndcg = []
relevance_score_list= []
for index, row in df_test.iterrows():
    expected_result= []
    query = literal_eval(row['Test_Query'])[:-2]
    query = preprocess(" ".join(query))
    num_indexes_to_shuffle = 3
    indexes_to_shuffle = random.sample(range(len(query)), num_indexes_to_shuffle)
    for index in indexes_to_shuffle:
        random.shuffle(query[index])
    expected_id = literal_eval(row['Recipe_IDs'])
    expected_result += [df.iloc[idx]['processed_ingredients'] for idx in expected_id]


    # Tokenize the query
    tokenized_query = query.split()
    retrieved_result = bm25.get_top_n(tokenized_query, pt, n=6)
    relevance_scores = [1 if i in expected_result else 0 for i in retrieved_result]
    relevance_score_list.append(relevance_scores)
    average_precisions.append(average_precision(relevance_scores))
    avg_ndcg.append(ndcg_at_k(relevance_scores,5))

# Compute the Mean Reciprocal Rank
map_score= np.mean(average_precisions)
mrr_score = mean_reciprocal_rank(relevance_score_list)
ndcg_score = np.mean(avg_ndcg)
print(f"MAP Score {map_score}")
print(f"NDCG Score {ndcg_score}")
print(f"Mean Reciprocal Rank Score: {mrr_score}")

MAP Score 0.9347222222222222
NDCG Score 0.9246623395562156
Mean Reciprocal Rank Score: 0.9731182795698925


## BM25 W QUERY EXPANSION

In [109]:
from collections import Counter
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [114]:
import ast
from ast import literal_eval
tokenized_corpus = [doc.split() for doc in pt]

bm25 = BM25Okapi(tokenized_corpus)

average_precisions = []
avg_ndcg = []
relevance_score_list= []

for index, row in df_test.iterrows():
    query = literal_eval(row['Test_Query'])
    query = preprocess(" ".join(query))
    num_indexes_to_shuffle = 3
    indexes_to_shuffle = random.sample(range(len(query)), num_indexes_to_shuffle)
    for index in indexes_to_shuffle:
        random.shuffle(query[index])
    expected_id = literal_eval(row['Recipe_IDs'])
    expected_result += [df.iloc[idx]['processed_ingredients'] for idx in expected_id]

    scores = bm25.get_scores(tokenized_query)

    tokenized_query = query.split()
    retrieved_result = bm25.get_top_n(tokenized_query, pt, n=5)

    top_docs_indices = np.argsort(scores)[::-1][:5]
    top_docs = [pt[i] for i in top_docs_indices]

    top_docs_tokens = [word_tokenize(doc) for doc in top_docs]
    top_docs_tokens = [token for doc in top_docs_tokens for token in doc]

    common_tokens = Counter(top_docs_tokens).most_common(5)
    expanded_query = tokenized_query + [token for token, freq in common_tokens]
    expanded_scores = bm25.get_scores(expanded_query)

    expanded_results_indices = np.argsort(expanded_scores)[::-1][:5]
    expanded_results = [pt[i] for i in expanded_results_indices]

    relevance_scores = [1 if i in expected_result else 0 for i in retrieved_result]
    average_precisions.append(average_precision(relevance_scores))
    relevance_score_list.append(relevance_scores)
    average_precisions.append(average_precision(relevance_scores))
    avg_ndcg.append(ndcg_at_k(relevance_scores,5))

# Compute the Mean Reciprocal Rank
map_score= np.mean(average_precisions)
mrr_score = mean_reciprocal_rank(relevance_score_list)
ndcg_score = np.mean(avg_ndcg)
print(f"MAP Score {map_score}")
print(f"NDCG Score {ndcg_score}")
print(f"Mean Reciprocal Rank Score: {mrr_score}")

MAP Score 0.9752688172043011
NDCG Score 0.9760858602692429
Mean Reciprocal Rank Score: 1.0


## COSINE SIMILARITY (doc2vec embedding)

In [116]:
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument

tagged_documents = [TaggedDocument(doc.split(), [i]) for i, doc in enumerate(pt)]

doc2vec_model = Doc2Vec(tagged_documents, dm=0, vector_size=100, window=2, min_count=1, workers=4)

In [122]:
average_precisions = []
avg_ndcg = []
relevance_score_list= []

for index, row in df_test.iterrows():
    query = literal_eval(row['Test_Query'])
    query = preprocess(" ".join(query))
    num_indexes_to_shuffle = 3
    indexes_to_shuffle = random.sample(range(len(query)), num_indexes_to_shuffle)
    for index in indexes_to_shuffle:
        random.shuffle(query[index])

    expected_id = literal_eval(row['Recipe_IDs'])
    expected_result += [df.iloc[idx]['processed_ingredients'] for idx in expected_id]
    tokenized_query = query.split()
    query_vector = doc2vec_model.infer_vector(tokenized_query)
    retrieved_vectors = doc2vec_model.dv.most_similar([query_vector], topn=5)
    indices = [doc_id for doc_id, similarity in retrieved_vectors]

    retrieved_result = [pt[i] for i in indices]
    relevance_scores = [1 if i in expected_result else 0 for i in retrieved_result]
    relevance_score_list.append(relevance_scores)
    average_precisions.append(average_precision(relevance_scores))
    avg_ndcg.append(ndcg_at_k(relevance_scores,5))

# Compute the Mean Reciprocal Rank
map_score= np.mean(average_precisions)
mrr_score = mean_reciprocal_rank(relevance_score_list)
ndcg_score = np.mean(avg_ndcg)
print(f"MAP Score {map_score}")
print(f"NDCG Score {ndcg_score}")
print(f"Mean Reciprocal Rank Score: {mrr_score}")


MAP Score 0.34193548387096767
NDCG Score 0.4120207168739592
Mean Reciprocal Rank Score: 0.36129032258064514


## COSINE SIMILARITY + BM 25 (DOC2VEC EMBEDDING)

In [127]:
average_precisions = []
avg_ndcg = []
relevance_score_list= []
for index, row in df_test.iterrows():
    query = literal_eval(row['Test_Query'])
    query = preprocess(" ".join(query))
    num_indexes_to_shuffle = 3
    indexes_to_shuffle = random.sample(range(len(query)), num_indexes_to_shuffle)
    for index in indexes_to_shuffle:
        random.shuffle(query[index])

    tokenized_query = query.split()
    query_vector = doc2vec_model.infer_vector(tokenized_query)
    retrieved_vectors = doc2vec_model.dv.most_similar([query_vector], topn=10)

    indices = [doc_id for doc_id, similarity in retrieved_vectors]

    intermediate_result = [pt[i] for i in indices]
    bm25 = BM25Okapi(intermediate_result)

    retrieved_result = bm25.get_top_n(tokenized_query, intermediate_result, n=5)

    relevance_scores = [1 if i in expected_result else 0 for i in retrieved_result]
    relevance_score_list.append(relevance_scores)
    average_precisions.append(average_precision(relevance_scores))
    avg_ndcg.append(ndcg_at_k(relevance_scores,5))

# Compute the Mean Reciprocal Rank
map_score= np.mean(average_precisions)
mrr_score = mean_reciprocal_rank(relevance_score_list)
ndcg_score = np.mean(avg_ndcg)
print(f"MAP Score {map_score}")
print(f"NDCG Score {ndcg_score}")
print(f"Mean Reciprocal Rank Score: {mrr_score}")


MAP Score 0.14247311827956988
NDCG Score 0.1858664357142876
Mean Reciprocal Rank Score: 0.14247311827956988


## COSINE SIMILARITY on TFIDF

In [154]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ast
from ast import literal_eval
import random
import numpy as np

# Add your definition of average_precision, ndcg_at_k, and preprocess functions here

# Assuming 'pt' and 'df_test' are defined somewhere in your code
df_test = pd.read_excel("/kaggle/input/validation/validation_set.xlsx")

# Example documents (recipes)
documents = list_of_recipe_words

average_precisions = []
avg_ndcg = []
relevance_score_list = []

# TF-IDF vectorization
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(documents)



for index, row in df_test.iterrows():
    expected_result = []
    query = literal_eval(row['Test_Query'])
    query = preprocess(" ".join(query))
    num_indexes_to_shuffle = 3
    indexes_to_shuffle = random.sample(range(len(query)), num_indexes_to_shuffle)

    for index in indexes_to_shuffle:
        random.shuffle(query[index])

    expected_id = literal_eval(row['Recipe_IDs'])
    expected_result += [df.iloc[idx]['processed_ingredients'] for idx in expected_id]
    # Vectorize user input
    user_vector = vectorizer.transform([query])
    # Calculate cosine similarity
    cosine_similarities = cosine_similarity(user_vector, tfidf_matrix).flatten()

    # Rank and retrieve top-n recipes
    top_n_indices = cosine_similarities.argsort()[::-1][:6]  # Assuming you want top 6 recipes
    retrieved_result = [documents[i] for i in top_n_indices]
    relevance_scores = [1 if i in expected_result else 0 for i in retrieved_result]
    relevance_score_list.append(relevance_scores)
    average_precisions.append(average_precision(relevance_scores))
    avg_ndcg.append(ndcg_at_k(relevance_scores, 5))

# Compute the Mean Reciprocal Rank
map_score = np.mean(average_precisions)
mrr_score = mean_reciprocal_rank(relevance_score_list)
ndcg_score = np.mean(avg_ndcg)

# Print the scores
print(f"MAP Score: {map_score}")
print(f"NDCG Score: {ndcg_score}")
print(f"Mean Reciprocal Rank Score: {mrr_score}")


MAP Score: 0.3758064516129032
NDCG Score: 0.42406499474548676
Mean Reciprocal Rank Score: 0.37741935483870964


## FAISS

In [163]:
! pip install numpy

In [ ]:
!pip install --upgrade dill==0.3.2
!pip install --upgrade pyarrow==3.0.0
!pip install --upgrade packaging==23.0
!pip install --upgrade overrides==6.0.1
!pip install --upgrade jupyter-lsp==2.0.0
!pip install --upgrade google-cloud-storage==2.2.1
!pip install --upgrade kubernetes==8.0.0
!pip install --upgrade opentelemetry-exporter-otlp-proto-grpc==1.19.0
!pip install --upgrade opentelemetry-exporter-otlp-proto-common==1.19.0
!pip install --upgrade opentelemetry-proto==1.19.0
!pip install --upgrade opentelemetry-sdk~=1.19.0
!pip install --upgrade packaging~=23.1


In [2]:
!pip -q install langchain huggingface_hub google-search-results tiktoken chromadb rank_bm25 faiss-cpu


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.7 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 14.0.1 which is incompatible.
google-cloud-bigquery 2.34.4 requires packaging<22.0dev,>=14.3, but you have packaging 23.2 which is incompatible.
google-cloud-pubsublite 1.8.3 requires overrides<7.0.0,>=6.0.1, but you have overrides 7.4.0 which is incompatible.
jupyterlab 4.0.5 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.0.1 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.0.1 requires jupyterlab<5.0.0a0,>=4.0.6, but you have jupyterlab 4.0.5 which is incompatible.
kfp 2.0.1 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-st

In [3]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.schema import Document

from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS

from gensim.models import Word2Vec

In [5]:
df['Instructions'] = df['Instructions'].str.replace('\n', ' ')


In [15]:
import glob,json

#Your desired template format
template = "Recipe id is {}. Recipe name is {}. Ingredients are {}. Instructions are {}"

# List to store the formatted strings
docs = []

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    # Extract values from each column
    recipe_id = row['Unnamed: 0.1']
    recipe_name = row['Title']
    ingredients = row['processed_ingredients']
    instructions = row['Instructions']

    # Create a formatted string using the template
    formatted_string = template.format(recipe_id, recipe_name, ingredients, instructions)
    formatted_dict = {
        'content': formatted_string,
        'id': recipe_id
    }

    # Convert the dictionary to a JSON string
    doc = formatted_string
    docs.append(formatted_string)

In [9]:
!pip install sentence-transformers

In [10]:
!pip install faiss-cpu


In [11]:
from langchain.embeddings import SentenceTransformerEmbeddings 

embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [31]:
faiss_vectorstore = FAISS.from_texts(docs, embeddings)
faiss_retriever = faiss_vectorstore.as_retriever(search_kwargs={"k": 4})

Batches:   0%|          | 0/422 [00:00<?, ?it/s]

In [76]:
s = faiss_retriever.get_relevant_documents("egg whites, new potatoes, kosher salt, finely ground black pepper, rosemary, thyme, parsley")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [77]:
s

[Document(page_content='Recipe id is 6360. Recipe name is Buttermilk Green Goddess Dressing. Ingredients are mayonnaise buttermilk fresh chive flat leaf parsley fresh tarragon fresh lemon juice anchovy fillet packed oil garlic kosher salt freshly ground black pepper. Instructions are In a processor, purée 1/2 cup mayonnaise, 1/3 cup buttermilk, 1/4 cup chopped fresh chives, 1/4 cup coarsely chopped flat-leaf parsley, 1 tablespoon chopped fresh tarragon, 1 tablespoon fresh lemon juice, 2 anchovy fillets packed in oil (drained, chopped), and 1 chopped garlic clove until smooth. Season with kosher salt and freshly ground black pepper. DO AHEAD: Can be made 2 hours ahead. Cover and chill.'),
 Document(page_content='Recipe id is 1. Recipe name is Crispy Salt and Pepper Potatoes. Ingredients are egg white new potato kosher salt finely ground black pepper rosemary thyme parsley. Instructions are Preheat oven to 400°F and line a rimmed baking sheet with parchment. In a large bowl, whisk the eg

In [78]:
import pandas as pd
import re

def retrieve_and_filter_df(df, s):
    """
    Retrieves and filters a DataFrame based on a list of items with 'page_content' attribute.

    Parameters:
    - df (pd.DataFrame): The original DataFrame.
    - s (list): A list of items where each item is an object with a 'page_content' attribute.

    Returns:
    - pd.DataFrame: The filtered DataFrame.
    """

    # Initialize an empty DataFrame to store the results
    rst = pd.DataFrame(columns=df.columns)
    retrieved_docs = []

    for idx, item in enumerate(s):
        # Extract the numeric ID from the 'page_content' attribute
        match = re.search(r'\b(\d+)\b', item.page_content)
        if match:
            value_to_find = int(match.group(1))

            # Use the value to filter rows in the DataFrame
            mask = df['Unnamed: 0.1'] == value_to_find
            rows_with_value = df[mask]
            rst = pd.concat([rst, rows_with_value])
            retrieved_docs.append(rst)

    return rst

# Example usage:
# Call the function with your DataFrame 'df' and the list 's'
        

      Unnamed: 0 Unnamed: 0.1                              Title  \
6360        6360         6360  Buttermilk Green Goddess Dressing   
1              1            1    Crispy Salt and Pepper Potatoes   
5614        5614         5614           Smoked Salmon Smørrebrød   
11894      11894        11894                       Winter Salad   

                                             Ingredients  \
6360   ['1/2 cup mayonnaise', '1/3 cup buttermilk', '...   
1      ['2 large egg whites', '1 pound new potatoes (...   
5614   ['1/2 cup mashed potatoes (optional)', '1/2 cu...   
11894  ['2 teaspoons finely chopped shallot', '1 teas...   

                                            Instructions  \
6360   In a processor, purée 1/2 cup mayonnaise, 1/3 ...   
1      Preheat oven to 400°F and line a rimmed baking...   
5614   Whisk potatoes, if using, sour cream, horserad...   
11894  Whisk together shallot, honey, orange juice, v...   

                                     Cleaned_Ingredients 

In [79]:
result_df = retrieve_and_filter_df(df, s)
result_df

,Unnamed: 0,Unnamed: 0.1,Title,Ingredients,Instructions,Cleaned_Ingredients,final_ing,processed_ingredients
6360,6360,6360,Buttermilk Green Goddess Dressing,"['1/2 cup mayonnaise', '1/3 cup buttermilk', '...","In a processor, purée 1/2 cup mayonnaise, 1/3 ...","['1/2 cup mayonnaise', '1/3 cup buttermilk', '...","['mayonnaise', 'buttermilk', 'fresh chives', '...",mayonnaise buttermilk fresh chive flat leaf pa...
1,1,1,Crispy Salt and Pepper Potatoes,"['2 large egg whites', '1 pound new potatoes (...",Preheat oven to 400°F and line a rimmed baking...,"['2 large egg whites', '1 pound new potatoes (...","['egg whites', 'new potatoes', 'kosher salt', ...",egg white new potato kosher salt finely ground...
5614,5614,5614,Smoked Salmon Smørrebrød,"['1/2 cup mashed potatoes (optional)', '1/2 cu...","Whisk potatoes, if using, sour cream, horserad...","['1/2 cup mashed potatoes (optional)', '1/2 cu...","['mashed potatoes', 'sour cream', 'fresh horse...",mashed potato sour cream fresh horseradish fre...
11894,11894,11894,Winter Salad,"['2 teaspoons finely chopped shallot', '1 teas...","Whisk together shallot, honey, orange juice, v...","['2 teaspoons finely chopped shallot', '1 teas...","['shallot', 'honey', 'fresh orange juice', 'sh...",shallot honey fresh orange juice sherry vinega...


In [85]:
import pandas as pd
import ast
from ast import literal_eval
import random
import numpy as np

# Add your definition of average_precision, ndcg_at_k, and preprocess functions here

df_test = pd.read_excel("/kaggle/input/validation/validation_set.xlsx")

average_precisions = []
avg_ndcg = []
relevance_score_list = []

for index, row in df_test.iterrows():
    expected_result = []
    query = literal_eval(row['Test_Query'])
    query =  ", ".join(query)

    expected_id = literal_eval(row['Recipe_IDs'])
    expected_result += [df.iloc[idx]['processed_ingredients'] for idx in expected_id]

    # Use faiss_retriever to get relevant documents
    s = faiss_retriever.get_relevant_documents(query)
    retriver_df =retrieve_and_filter_df(df, s)

    # Assuming 's' is a list of items where each item is an object with a 'page_content' attribute
    retrieved_result = [item for item in retriver_df['processed_ingredients']]
    
    
    relevance_scores = [1 if i in expected_result else 0 for i in retrieved_result]
    relevance_score_list.append(relevance_scores)
    average_precisions.append(average_precision(relevance_scores))
    avg_ndcg.append(ndcg_at_k(relevance_scores, 5))

# Compute the Mean Reciprocal Rank
map_score = np.mean(average_precisions)
mrr_score = mean_reciprocal_rank(relevance_score_list)
ndcg_score = np.mean(avg_ndcg)

# Print the scores
print(f"MAP Score: {map_score}")
print(f"NDCG Score: {ndcg_score}")
print(f"Mean Reciprocal Rank Score: {mrr_score}")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

MAP Score: 0.5698924731182795
NDCG Score: 0.6273030525921673
Mean Reciprocal Rank Score: 0.5752688172043011


## ENSEMBLE RETRIVER 

In [91]:
# initialize the bm25 retriever and faiss retriever
bm25_retriever = BM25Retriever.from_texts(pt)
bm25_retriever.k = 5

In [87]:
bm25_retriever.get_relevant_documents("Generapte recipe using whole chicken, kosher salt, acorn squash, sage")

[Document(page_content='delicata acorn squash garlic sage thyme extra virgin olive oil kosher salt white wine vinegar'),
 Document(page_content='acorn squash extra virgin olive oil fresh nutmeg honey sage leaf salt freshly ground black pepper'),
 Document(page_content='fresh hatch chile unsalted butter onion garlic using microplane kosher salt purpose flour ground cumin whole milk cheddar cheese monterey jack cheese plum tomato cilantro white corn tortilla chip'),
 Document(page_content='buttery sugar cookie dough recipe dulce de leche cream cheese sugar egg pure vanilla extract'),
 Document(page_content='using electric mixer powdered sugar egg white minute add lemon juice divide icing add different food coloring')]

In [88]:
# initialize the ensemble retriever
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, faiss_retriever],
                                       weights=[0.4, 0.8])

In [89]:
dc = ensemble_retriever.get_relevant_documents("whole chicken, kosher salt, acorn squash sage")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [93]:
dc

[Document(page_content='Recipe id is 0. Recipe name is Miso-Butter Roast Chicken With Acorn Squash Panzanella. Ingredients are whole chicken kosher salt acorn squash sage rosemary unsalted butter ground allspice crushed red pepper flake freshly ground black pepper white bread apple extra virgin olive oil red onion apple cider vinegar white miso purpose flour unsalted butter dry white wine unsalted chicken broth white miso kosher salt freshly ground pepper. Instructions are Pat chicken dry with paper towels, season all over with 2 tsp. salt, and tie legs together with kitchen twine. Let sit at room temperature 1 hour. Meanwhile, halve squash and scoop out seeds. Run a vegetable peeler along ridges of squash halves to remove skin. Cut each half into ½"-thick wedges; arrange on a rimmed baking sheet. Combine sage, rosemary, and 6 Tbsp. melted butter in a large bowl; pour half of mixture over squash on baking sheet. Sprinkle squash with allspice, red pepper flakes, and ½ tsp. salt and seas

In [98]:
import pandas as pd
import re

def retrieve_and_filter_df_en(df, s):
    """
    Retrieves and filters a DataFrame based on a list of items with 'page_content' attribute.

    Parameters:
    - df (pd.DataFrame): The original DataFrame.
    - s (list): A list of items where each item is an object with a 'page_content' attribute.

    Returns:
    - pd.DataFrame: The filtered DataFrame.
    """

    # Initialize an empty DataFrame to store the results
    rst = pd.DataFrame(columns=df.columns)
    retrieved_docs = []

    for idx, item in enumerate(s):
        # Extract the numeric ID from the 'page_content' attribute
        item = item.page_content
        if item[0:6]=="Recipe":
            match = re.search(r'\b(\d+)\b', item)
            if match:
                value_to_find = int(match.group(1))

                # Use the value to filter rows in the DataFrame
                mask = df['Unnamed: 0.1'] == value_to_find
                rows_with_value = df[mask]
                rst = pd.concat([rst, rows_with_value])
                retrieved_docs.append(rst)
        else: 
            for index, row in df.iterrows():
        # Check if the match_string is present in any column of the current row
                if any(item.lower() in str(value).lower() for value in row):
                    rst = pd.concat([rst, pd.DataFrame(row).transpose()], ignore_index=True)

            

    return rst
retrieve_and_filter_df_en(df,dc)

,Unnamed: 0,Unnamed: 0.1,Title,Ingredients,Instructions,Cleaned_Ingredients,final_ing,processed_ingredients
0,0,0,Miso-Butter Roast Chicken With Acorn Squash Pa...,"['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","Pat chicken dry with paper towels, season all ...","['1 (3½–4-lb.) whole chicken', '2¾ tsp. kosher...","['whole chicken', 'kosher salt', 'acorn squash...",whole chicken kosher salt acorn squash sage ro...
1,10178,10178,Butternut Squash and Sage Soup with Sage Bread...,"['1 1/2 tablespoons butter', '1 1/2 tablespoon...",Melt butter with oil in large pot over medium-...,"['1 1/2 tablespoons butter', '1 1/2 tablespoon...","['butter', 'olive oil', 'onions', 'fresh itali...",butter olive oil onion fresh italian parsley f...
2,5320,5320,"Roasted Poultry, Wild Boar Bacon, and Mushroom...","['Bones from 4 whole game hens, small chickens...","In a large stockpot, combine the bones, water,...","['Bones from 4 whole game hens, small chickens...","['whole game hens, chickens', 'water', 'carrot...",whole game hen chicken water carrot onion cele...
3,1022,1022,Slow-Cooked Winter Squash with Sage and Thyme,"['1 lb. delicata or acorn squash, halved lengt...",Place a rack in middle of oven and preheat to ...,"['1 lb. delicata or acorn squash, halved lengt...","['delicata or acorn squash', 'garlic', 'sage',...",delicata acorn squash garlic sage thyme extra ...
4,1022,1022,Slow-Cooked Winter Squash with Sage and Thyme,"['1 lb. delicata or acorn squash, halved lengt...",Place a rack in middle of oven and preheat to ...,"['1 lb. delicata or acorn squash, halved lengt...","['delicata or acorn squash', 'garlic', 'sage',...",delicata acorn squash garlic sage thyme extra ...
5,1469,1469,Roasted Acorn Squash and Honey,"['2 acorn squash (about 1 pound each)', '2 tab...",Preheat the oven to 350°F. Split the squash in...,"['2 acorn squash (about 1 pound each)', '2 tab...","['acorn squash', 'extra-virgin olive oil', 'fr...",acorn squash extra virgin olive oil fresh nutm...
6,12974,12974,"Roasted Squash, Chestnut, and Chicory Salad wi...",['2 tablespoons extra-virgin olive oil plus ad...,Put oven rack in middle position and preheat o...,['2 tablespoons extra-virgin olive oil plus ad...,"['extra-virgin olive oil', 'acorn squash', 'sa...",extra virgin olive oil acorn squash salt black...
7,12219,12219,"Dandelion Salad with Pomegranate Seeds, Pine N...","['6 tablespoons pomegranate juice', '1 1/2 tab...",Whisk pomegranate juice and vinegars in bowl. ...,"['6 tablespoons pomegranate juice', '1 1/2 tab...","['pomegranate juice', 'balsamic vinegar', 'red...",pomegranate juice balsamic vinegar red wine vi...
8,342,342,Steamed Kabocha With Ginger-Soy Dressing,"['3 scallions, white and dark green parts sepa...","Pour water into a large saucepan, wide skillet...","['3 scallions, white and dark green parts sepa...","['scallions', 'garlic', 'seasoned rice vinegar...",scallion garlic seasoned rice vinegar sesame o...


In [99]:
import pandas as pd
import ast
from ast import literal_eval
import random
import numpy as np

# Add your definition of average_precision, ndcg_at_k, and preprocess functions here

df_test = pd.read_excel("/kaggle/input/validation/validation_set.xlsx")

average_precisions = []
avg_ndcg = []
relevance_score_list = []

for index, row in df_test.iterrows():
    expected_result = []
    query = literal_eval(row['Test_Query'])
    query =  ", ".join(query)

    expected_id = literal_eval(row['Recipe_IDs'])
    expected_result += [df.iloc[idx]['processed_ingredients'] for idx in expected_id]

    # Use faiss_retriever to get relevant documents
    s = ensemble_retriever.get_relevant_documents(query)
    retriver_df =retrieve_and_filter_df_en(df, s)

    # Assuming 's' is a list of items where each item is an object with a 'page_content' attribute
    retrieved_result = [item for item in retriver_df['processed_ingredients']]
    
    
    relevance_scores = [1 if i in expected_result else 0 for i in retrieved_result]
    relevance_score_list.append(relevance_scores)
    average_precisions.append(average_precision(relevance_scores))
    avg_ndcg.append(ndcg_at_k(relevance_scores, 5))

# Compute the Mean Reciprocal Rank
map_score = np.mean(average_precisions)
mrr_score = mean_reciprocal_rank(relevance_score_list)
ndcg_score = np.mean(avg_ndcg)

# Print the scores
print(f"MAP Score: {map_score}")
print(f"NDCG Score: {ndcg_score}")
print(f"Mean Reciprocal Rank Score: {mrr_score}")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

MAP Score: 0.48738351254480267
NDCG Score: 0.5567851610302778
Mean Reciprocal Rank Score: 0.6462365591397847
